#Regressão -Preparação de features
---
**Aula Prática 02**: Preparamento de features


**Objetivo**Preparar features para serem treinadas em um modelo.


Banco de dados:


Preço de carros usados


[Disponivel no kaggle](https://www.kaggle.com/datasets/rishabhkarn/used-car-dataset/data)


[Disponível para download](https://drive.google.com/file/d/1Ny6GypPH4AtJi6CJHmEUEI3KN11hDuGG/view?usp=drive_link)


Descrição dos dados:
* car_name: nome do carro
* registration_year: ano de registro
* insurance_validity: tipo de seguro
* fuel_type: tipo de combustível
* seats: número de assentos
* kms_drive: total km dirigidos
* ownsership: número de proprietários
* transmission: tipo de câmbio
* manufacturing_year: ano de fabricação
* mileage(kmpl): km por litro
* engine (cc): tamanho do motor
* max_power(bhp): potência do carro
* torque(Nm): torque do motor
* price (in lakhs): preço em lakhs (medida indiana para 100.000)

##Import das principais funções e leitura dos dados


---


In [ ]:
import pandas as pd #pacote para leitura dos dados
import numpy as np

In [ ]:
#opção 1 -> montar o drive no colab e acessar o arquivo
#from google.colab import drive
#drive.mount('/content/drive')


#opção 2 -> fazer download e fazer upload por aqui
from google.colab import files
uploaded = files.upload()

In [ ]:
path = 'Used Car Dataset.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.rename(columns = {'mileage(kmpl)':'mileage',
                          'engine(cc)':'engine',
                          'max_power(bhp)': 'max_power',
                          'torque(Nm)': 'torque',
                          'price(in lakhs)': 'price'})

df.drop(columns='Unnamed: 0', inplace=True)

## Preparação de dados numéricos
---
Para o processamento das variáveis numéricas faremos o seguinte:

- Imputação/exclusão em dados faltantes

Para realizar isso utilizaremos a função `fillna`.

```python
# Preencher dado faltante
df['coluna'].fillna(valor)
```

**Exercícios**:

Preencha as variáveis que possuem dados faltantes com a mediana das observações.

In [ ]:
df.dtypes

In [ ]:
var_numerica = ['seats', 'kms_driven', 'mileage', 'engine', 'max_power', 'torque']

## Preparação de dados categóricos

---
Para o processamento das variáveis categóricas iremos transformar as categorias em variáveis numéricas, conhecidas como variáveis dummy.


---
### Variáveis Dummy

Variáveis Dummy é uma representação de uma variável categórica com k categorias em k-1 variáveis, todas as novas variáveis são binárias.


Exemplo:
Variável faixa de idade, temos três categorias de idades entre 0-18, entre 18-30, entre 30-50 e 50+.
Na tabela abaixo na coluna variável original temos a categoria como estava preenchida no campo original, as demais três colunas são as novas variáveis criadas no banco de dados.


|Variável original|18-30|30-50|50+|
|---|---|---|---|
|0-18|0|0|0|
|18-30|1|0|0|
|30-50|0|1|0|
|50+|0|0|1|


Obs.: No exemplo acima a categoria entre 0-18 é a categoria **base**.

Para criar variáveis dummy iremos utilizar a função get_dummy do pacote Pandas


Exemplo:
``` python
import pandas as pd
df_dummy = pd.get_dummies(df, drop_first=True)
```
Exercícios:



Crie uma variável dummy para o conjunto de dados? Notou algum problema na criação das dummys?


In [ ]:
df.dtypes

In [ ]:
var_cat = ['registration_year',
           'fuel_type',
           'ownsership',
           'transmission']

In [ ]:
var_cat + var_numerica

In [ ]:
df[var_cat + var_numerica]

### Limpeza das categorias

Iremos agora fazer um processo de limpeza de categorias que foram preenchidas de forma incorreta.

1. A variável `registration_year`, ela possui campos preenchidos de forma incorreta, o padrão é o ano ou mês-ano;

Para processar essa categoria iremos criar uma função que:
1. Checa se o tamanho da variável é diferente de 4, isso é foi preenchido com algo diferente do ano;
2. Se é diferente de 4, iremos quebrar a string com - e pegar o ano;
3. Caso a operação 2 não ocorrerá será retornado o número 200000

Após essa limpeza iremos criar três categorias, carros com registro antes de 2010, entre 2010 e 2020 e após 2020.

In [ ]:
var_cat[0]

In [ ]:
df[var_cat[0]].value_counts()

In [ ]:
def gera_data(registration_year):
  if len(registration_year)!=4:
    try:
      return '20' + registration_year.split('-')[1]
    except:
      return '200000'
  return registration_year

In [ ]:
df[var_cat[0]].apply(gera_data).value_counts()

In [ ]:
df['registration_tratada'] = df[var_cat[0]].apply(gera_data).astype(int)

In [ ]:
def categoria_idade(registration_year):
  if registration_year < 2010:
    return 'antes_2010'
  elif registration_year < 2020:
    return 'antes 2020'
  elif registration_year < 2030:
    return 'depois 2020'
  else:
    return 'None'

In [ ]:
df['registration_tratada'].apply(categoria_idade).value_counts(dropna=False)

In [ ]:
df['registration_tratada'] = df['registration_tratada'].apply(categoria_idade)

In [ ]:
df['registration_tratada']

2. Limpeza da variável `fuel type`

Iremos remover as linhas preenchidas com o valor '5 Seats'

In [ ]:
var_cat[1]

In [ ]:
df[var_cat[1]].value_counts(dropna=False)

In [ ]:
df = df[df[var_cat[1]] != '5 Seats']

In [ ]:
df[var_cat[1]].value_counts()

In [ ]:
df['registration_tratada'].value_counts(dropna=False)

3. Limpeza da variável `ownsership`

Iremos manter apenas as categorias 'First Owner, 'Second Owner', 'Third Owner' e 'Fifth Owner'

In [ ]:
df[var_cat[2]].value_counts()

In [ ]:
def tratamento_ownsership(ownsership):
  if ownsership in ['First Owner', 'Second Owner', 'Third Owner', 'Fifth Owner']:
    return ownsership
  else:
    return 'None'

In [ ]:
df['ownsership_tratado'] = df.ownsership.apply(tratamento_ownsership)

In [ ]:
df['ownsership_tratado'].value_counts()

4. Limpeza da variável `transmission`

Iremos manter apenas as categorias 'Manual' e 'Automatic'

In [ ]:
df[var_cat[3]].value_counts()

In [ ]:
df['transmission_tratado'] = df[var_cat[3]].apply(lambda x: x if x in ['Manual', 'Automatic'] else 'None')

In [ ]:
df['transmission_tratado'].value_counts()

In [ ]:
var_cat = ['registration_tratada',
           'fuel_type',
           'ownsership_tratado',
           'transmission_tratado']

In [ ]:
df_dummy = pd.get_dummies(df[var_cat + var_numerica],drop_first=True)

In [ ]:
df_dummy.head()

In [ ]:
var_numerica.append('price')
df_dummy = pd.get_dummies(df[var_cat + var_numerica],drop_first=True)
df_dummy.to_csv('dado_tratado.csv', index=False)

In [ ]:
df_dummy

### Variáveis one hot

Variáveis one hot encoding é uma representação de uma variável categórica com k categorias em k variáveis, todas as novas variáveis são binárias.

Exemplo:
Variável faixa de idade

|Variável original|0-18|18-30|30-50|50+|
|---|---|---|---|---|
|0-18|1|0|0|0|
|18-30|0|1|0|0|
|30-50|0|0|1|0|
|50+|0|0|0|1|

Para criar variáveis dummy iremos utilizar a função get_dummy do pacote Pandas

Exemplo:
``` python
import pandas as pd
df_dummy = pd.get_dummies(df)
```

Exercícios:

Crie uma variável dummy com representação one-hot encoding para o dado tratado.